## Load Match from Statsbomb

In [ ]:
from kloppy import statsbomb
import polars as pl

dataset = statsbomb.load_open_data(
    match_id=15946,
    coordinates="statsbomb"
)

c:\Users\aless\Documents\GitHub\kloppy\kloppy\_providers\statsbomb.py:83: UserWarning: 

You are about to use StatsBomb public data.
By using this data, you are agreeing to the user agreement. 
The user agreement can be found here: https://github.com/statsbomb/open-data/blob/master/LICENSE.pdf

  warnings.warn(


## Extract Minutes Played

In [6]:
mins_played = dataset.aggregate("minutes_played")

for item in mins_played:
    print(f"{item.player} - {item.duration.total_seconds() / 60:.1f} minutes_played")

Philippe Coutinho Correia - 47.5 minutes_played
Sergio Busquets i Burgos - 84.3 minutes_played
Jordi Alba Ramos - 92.6 minutes_played
Gerard Piqué Bernabéu - 92.6 minutes_played
Luis Alberto Suárez Díaz - 92.6 minutes_played
Ivan Rakitić - 92.6 minutes_played
Ousmane Dembélé - 76.3 minutes_played
Samuel Yves Umtiti - 92.6 minutes_played
Lionel Andrés Messi Cuccittini - 92.6 minutes_played
Nélson Cabral Semedo - 45.1 minutes_played
Sergi Roberto Carnicer - 92.6 minutes_played
Arturo Erasmo Vidal Pardo - 8.3 minutes_played
Arthur Henrique Ramos de Oliveira Melo - 16.3 minutes_played
Marc-André ter Stegen - 92.6 minutes_played
Borja González Tomás - 24.6 minutes_played
Jonathan Rodríguez Menéndez - 68.0 minutes_played
Rubén Duarte Sánchez - 92.6 minutes_played
Rubén Sobrino Pozuelo - 70.6 minutes_played
Víctor Laguardia Cisneros - 92.6 minutes_played
Ibai Gómez Pérez - 92.6 minutes_played
Martín Aguirregabiria Padilla - 92.6 minutes_played
Jorge Franco Alviz - 22.0 minutes_played
Mubarak 

## Compute Passes per 90 Minutes
Filter passes off the datasets and convert to Polars dataframe. Calculate minutes played by each player

In [9]:
passes_polars = dataset.filter("Pass").to_df(
    "player_id",
    lambda event: {
        "player_name": str(event.player),
        "successful_passes": event.result.is_success if event.result is not None else None
    },
    engine="polars"
)


mins_played_pl = pl.DataFrame(
    [
        {
            "player_id": item.player.player_id,
            "minutes_played": item.duration.total_seconds() / 60
        }
        for item in mins_played
    ]
)

Now, to calculate the p90 metrics:

- Group by player_id and player_name to aggregate pass statistics.
- Compute successful passes and total passes.
- Join with mins_played_pl to include minutes played.
- Calculate Per 90 Metrics:
    - success_p90: Successful passes per 90 minutes.
    - total_p90: Total passes per 90 minutes.

In [11]:
passes_p90 = (
    passes_polars.group_by(["player_id", "player_name"])
    .agg(successful_passes=pl.col("successful_passes").sum(),total_passes=pl.col("successful_passes").count())
    .join(mins_played_pl, on="player_id")
    .with_columns(
        success_p90=(pl.col("successful_passes") / pl.col("minutes_played")) * 90,
        total_p90=(pl.col("total_passes") / pl.col("minutes_played")) * 90
    )
)

passes_p90

player_id,player_name,successful_passes,total_passes,minutes_played,success_p90,total_p90
str,str,u32,u32,f64,f64,f64
"""3501""","""Philippe Coutinho Correia""",46,52,47.517683,87.12546,98.48965
"""5203""","""Sergio Busquets i Burgos""",77,83,84.319283,82.187606,88.591835
"""5211""","""Jordi Alba Ramos""",117,128,92.618717,113.691923,124.380907
"""5213""","""Gerard Piqué Bernabéu""",76,81,92.618717,73.851164,78.709793
"""5246""","""Luis Alberto Suárez Díaz""",28,39,92.618717,27.208323,37.897308
…,…,…,…,…,…,…
"""6629""","""Fernando Pacheco Flores""",12,25,92.618717,11.66071,24.293146
"""6632""","""Manuel Alejandro García Sánche…",16,20,92.618717,15.547613,19.434517
"""6839""","""Daniel Alejandro Torres Rojas""",12,16,67.909867,15.903433,21.204577
